In [ ]:
import glob
from deliverable.main.tsp.TSP_Formulation_Methods import *
import matplotlib.pyplot as plt
import numpy as np
import os

N = 5
p = 2
num_reads_solver = 4000

script_dir = os.getcwd()
concrete_simulations = "lambdas_N_"
files_to_find = os.path.join(script_dir, f"data/lamdasOptimized/{concrete_simulations}*")
list_of_files = glob.glob(files_to_find)

mean_lambdas = load_lambda_means(list_of_files)

print("Mean lambdas: ", mean_lambdas)
distances_original_matrix = np.loadtxt("./data/matriz-rutas-granada")
all_start_end_combinations = generate_all_start_end_combinations(N, L=1)
distances_N_stops_normalized = distances_original_matrix[:N,:N]/np.max(distances_original_matrix[:N,:N])

In [ ]:
files_to_find = "./data/lamdasOptimized/"+concrete_simulations+"*"
files = glob.glob(files_to_find)

all_weights = []

for file in files:
    weights = np.loadtxt(file)
    all_weights.append(weights)

all_weights = np.array(all_weights)  # Shape: (num_files, 5)

means = np.mean(all_weights, axis=0)
variances = np.var(all_weights, axis=0)

for i in range(5):
    print(f"Weight λ{i+1}: Mean = {means[i]:.4f}, Variance = {variances[i]:.4f}")

plt.figure(figsize=(10, 5))
for i in range(5):
    plt.hist(all_weights[:, i], bins=10, alpha=0.6, label=f"λ{i+1}")
plt.xlabel("Valor del peso")
plt.ylabel("Frecuencia")
plt.title("Distribución de los pesos óptimos")
plt.legend()
plt.show()

In [ ]:
plt.boxplot(all_weights, labels=[f"λ{i+1}" for i in range(5)])
plt.ylabel("Weight value")
plt.title("Variability of optimal weights")
plt.show()

In [ ]:
# Check on how accurate are the mean weights for each combination of start and end stops

solutions_analysis = np.zeros((len(all_start_end_combinations), 3))

for index, (startNodes, endNodes) in enumerate(all_start_end_combinations):
    startNode = startNodes[0]
    endNode = endNodes[0]
    solutions_analysis[index][0] = startNode
    solutions_analysis[index][1] = endNode
    Q_matrix,_ = create_QUBO_matrix(distances_N_stops_normalized, p, startNode, endNode, mean_lambdas)
    solution_array, _ = solve_qubo_with_Dwave(Q_matrix, num_reads_solver)
    solutions_analysis[index][2] = int(check_solution_return(solution_array, N, p, startNode, endNode))

show_statistics_lambdas(solutions_analysis)

check_solution(solution_array, N, p, N-2, N-1)
draw_solution_graph(solution_array, distances_N_stops_normalized, p, solutions_analysis[-1][0], solutions_analysis[-1][1])
